In [25]:
import boto3
import json

# Use your profile directly
session = boto3.Session(profile_name="bedrock-access-user", region_name="us-east-1")

# Create SageMaker runtime client
sagemaker_runtime_client = session.client("sagemaker-runtime")

# Define endpoint name
endpoint_name = "huggingface-pytorch-inference-2025-08-26-09-59-59-999"

# Prepare prompt
system_prompt = """
You are a helpful assistant that answers questions about the capitals of countries.

Your output should be in a structured JSON format exactly like the one below. You are not allowed to write anything other than JSON object:

[
    {
        "country": the country that you will get the capital of,
        "capital": the capital of the country stated
    }
]
"""

messages = [
    {"role": "system", "content": system_prompt},
]

messages.append({"role": "user", "content": "Get me the capitals of following countries: India, France, Germany, The USA, Russia, China"})

# Example request body
payload = json.dumps({
    "inputs": messages,
    "parameters": {
        "max_new_tokens": 500,   
        "temperature": 0.7,
        "top_p": 0.9,     
        "do_sample": True,
        "return_full_text": False
    }
})

# Invoke the endpoint
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# Read the response
result = response["Body"].read().decode("utf-8")
parsed = json.loads(result)

# Extract generated text safely
if isinstance(parsed, list) and "generated_text" in parsed[0]:
    output_text = parsed[0]["generated_text"]
else:
    output_text = parsed

# Clean up output: remove extra formatting if any
# output_text = output_text.strip()

print(output_text)


 [
    {
        "country": "India",
        "capital": "New Delhi"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "The USA",
        "capital": "Washington, D.C."
    },
    {
        "country": "Russia",
        "capital": "Moscow"
    },
    {
        "country": "China",
        "capital": "Beijing"
    }
]
